In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from shapely.geometry import Point, Polygon
import warnings; warnings.filterwarnings("ignore")

In [ ]:
puntos_especies = pd.read_csv("..\\data\\observacion_puntos\\especies.csv")
cuadrantes_especie = pd.read_csv("..\\data\\observacion_puntos\\cuadrantes_especie.csv")

In [ ]:
puntos_especies.head()

In [ ]:
cuadrantes_especie.head()

In [ ]:
## Separamos los datos
filtro_cuadrante1 = cuadrantes_especie["cuadrante"] == "Antes"
filtro_cuadrante2 = cuadrantes_especie["cuadrante"] == "Despues"

df_cuadrantes_antes = cuadrantes_especie[filtro_cuadrante1]
df_cuadrantes_despues = cuadrantes_especie[filtro_cuadrante2]

## Crear una capa SHP de puntos, marcando las especies encontradas

In [ ]:
geometria_puntos = [Point(xy) for xy in zip(puntos_especies["latitud"], puntos_especies["longitud"])]
gdf_puntos_especie = gpd.GeoDataFrame(puntos_especies, geometry=geometria_puntos, crs="epsg:32628")
gdf_puntos_especie.head()

In [ ]:
## Etiquetas
gdf_puntos_especie["etiquetas"] = gdf_puntos_especie["especie"] + "\n" + gdf_puntos_especie["fecha"]
gdf_puntos_especie

## Crear una capa SHP con los polígonos para mostrar la extensión de tierra de las especies

Primero hay que crear una capa de puntos como el apartado anterior

In [ ]:
geometria_cuadrantes_antes = [Point(xy) for xy in zip(df_cuadrantes_antes["latitud"], df_cuadrantes_antes["longitud"])]
gdf_puntos_cuadrantes_antes = gpd.GeoDataFrame(df_cuadrantes_antes, geometry=geometria_cuadrantes_antes, crs="epsg:32628")

geometria_cuadrantes_despues = [Point(xy) for xy in zip(df_cuadrantes_despues["latitud"], df_cuadrantes_despues["longitud"])]
gdf_puntos_cuadrantes_despues = gpd.GeoDataFrame(df_cuadrantes_despues, geometry=geometria_cuadrantes_despues, crs="epsg:32628")

In [ ]:
gdf_puntos_cuadrantes_antes.head()

In [ ]:
gdf_puntos_cuadrantes_despues

Pero esta vez además hay que unirlo para dar lugar a polígonos

In [ ]:
poligono_cuadrante1 = Polygon(gdf_puntos_cuadrantes_antes["geometry"])
poligono_cuadrante2 = Polygon(gdf_puntos_cuadrantes_despues["geometry"])


In [ ]:
## Creamos el data frame
gdf_cuadrantes_poligonos = gpd.GeoDataFrame({"cuadrante": cuadrantes_especie["cuadrante"].unique(), 
                                             "geometry": [poligono_cuadrante1, 
                                                          poligono_cuadrante2]})
print(gdf_cuadrantes_poligonos)

In [ ]:
fig, ax = plt.subplots(1,1)

gdf_cuadrantes_poligonos.plot(ax = ax, 
                              column="cuadrante", 
                              alpha = .5,
                              cmap=ListedColormap(["red", "blue"]),
                              legend=True)
gdf_puntos_especie.plot(ax = ax, edgecolor="black", facecolor="yellow")

## Descargamos las capas como archivos SHP

In [ ]:
gdf_puntos_especie.to_file("..\\data\\gran_canaria\\puntos_especie.shp")


In [ ]:
gdf_cuadrantes_poligonos.to_file("..\\data\\gran_canaria\\poligonos_especie.shp")